### Calculate Experiment Analyses Values Per Larvae
#### List of tasks accomplished in this Jupyter Notebook:
- Calculate the median concentration preferred by the larvae (normalized to the acclimation period)
- Calculate the discovery time for each larvae (normalized to the acclimation period)
- Calculate the speed in areas of high concentration, compared to speed in areas of low concentration, normalized to behavior during the acclimation period. 
- Output all of these values into a separate spreadsheet. 

In [1]:
import numpy as np
import pandas as pd
import math, os
import more_itertools as mit

In [2]:
concentration_cutoff = 50

fps = 2
sh = 1 # direction in which to shift vector for delta calculations
threshold = 2 # consecutive seconds spent moving
still_threshold = 10 # seconds for minimum still num count
angle_thresh = 20 # Allowed error from previous angle
sharp_angle = 45
spiral_thresh = 4
spiral_speed_thresh = np.inf

In [3]:
def safe_divide(x, y):
    x = safe_get(x)
    y = safe_get(y)
    try: 
        ans = x / y
    except ZeroDivisionError: 
        return 0
    return ans
    
def safe_get(x, ans=0):
    if math.isnan(x):
        return ans
    return x

In [4]:
df = pd.read_csv("./data/experiment_IDs/cleaned_static_data.csv")

animals = df["animal_ID"].unique()
# Check that animal IDs in dataframe are unique
assert len(df) == len(animals)

fps = 2
a_master_df = pd.DataFrame()
e_master_df = pd.DataFrame()

for index, row in df.iterrows():
    animal = row['animal_ID']
    aID = animal[:9]
    pos = animal[10:]
    
    exp_filename = "./data/trajectories/video_calculations/"+aID+'-E-'+pos+'.csv'
    acc_filename = "./data/trajectories/video_calculations/"+aID+'-A-'+pos+'.csv'
    
    for tag, filename, master_df in zip(['A_', 'E_'], 
                             [acc_filename, exp_filename], 
                             [a_master_df, e_master_df]):
        if os.path.isfile(filename):
            temp = pd.read_csv(filename)

            # MEDIAN CONCENTRATION -----------------------------
            median_conc = safe_get(temp["concentration"].median())

            # DISCOVERY TIME IN MINUTES ------------------------
            if len(temp[temp["concentration"] >= concentration_cutoff]) > 0:
                discovery_time = safe_divide(temp[temp["concentration"] >= concentration_cutoff]["frames"].values[0], fps*60)
            else:
                discovery_time = 15

            # Divide up into thresholds for concentration and concentration differences
            high = temp[temp["concentration"] >= 50]
            low = temp[temp["concentration"] < 50]
            high_move = high[high["moving"] == True]
            low_move = low[low["moving"] == True]
            
            # SPEED IN BODY LENGTHS ----------------------------
            # concentration dependent
            high_speed = safe_divide(high_move["speed_BL"].median(), row["larvae_length_mm"])
            low_speed = safe_divide(low_move["speed_BL"].median(), row["larvae_length_mm"])
            c_speed = high_speed - low_speed
 
            temp = pd.DataFrame({# STATIC VARIABLES --------------------------
                                "animal_ID": [row["animal_ID"]], 
                                "treatment_odor": row["treatment_odor"],
                                "sex": row["sex"], 
                                "species": row['species'],
                                'dead':row['dead'],

                                # VARIABLES CALCULATED FOR ENTIRE TIME -------
                                tag+"median_conc": median_conc, 
                                tag+"discovery_time": discovery_time,
                                tag+"c_speed": c_speed
                                 })
            if tag == 'A_':
                a_master_df = pd.concat([a_master_df, temp])
            elif tag == 'E_':
                e_master_df = pd.concat([e_master_df, temp])

display(a_master_df.head(2))
display(e_master_df.head(2))
a_master_df.drop(['treatment_odor', 'sex', 'species', 'dead'], axis=1, inplace=True)
df = pd.merge(a_master_df, e_master_df, on='animal_ID', how='inner')

df["median_conc_diff"] = df["E_median_conc"] - df["A_median_conc"]
df["discovery_time_diff"] = df["E_discovery_time"] - df["A_discovery_time"] 
df["c_speed_diff"] = df["E_c_speed"] - df["A_c_speed"]

df.to_csv('./data/trajectories/cleaned_animal_analyses_experiment.csv', index=False)
display(df.tail())

,animal_ID,treatment_odor,sex,species,dead,A_median_conc,A_discovery_time,A_c_speed
0,190626-01-top,100ul_milliQ_water,f,Aedes aegypti,no,4.957667,0.075000,-0.013474
0,190626-01-bottom,100ul_milliQ_water,f,Aedes aegypti,no,6.024708,0.833333,-0.058355


,animal_ID,treatment_odor,sex,species,dead,E_median_conc,E_discovery_time,E_c_speed
0,190626-01-top,100ul_milliQ_water,f,Aedes aegypti,no,15.134000,0.750000,-0.008422
0,190626-01-bottom,100ul_milliQ_water,f,Aedes aegypti,no,15.773792,0.016667,-0.070911


,animal_ID,A_median_conc,A_discovery_time,A_c_speed,treatment_odor,sex,species,dead,E_median_conc,E_discovery_time,E_c_speed,median_conc_diff,discovery_time_diff,c_speed_diff
500,191130-06-bottom,8.555708,4.125000,-0.013755,100ul_quinine,m,Culex tarsalis,no,8.841000,1.050000,0.003728,0.285292,-3.075000,0.017483
501,191130-07-top,4.530250,2.891667,-0.012788,05_percent_food,m,Culex tarsalis,no,23.201625,0.841667,-0.032818,18.671375,-2.050000,-0.020030
502,191130-08-top,14.006667,1.858333,-0.007765,05_percent_food,f,Culex tarsalis,no,24.578833,0.016667,-0.009597,10.572167,-1.841667,-0.001831
503,191130-08-bottom,2.001667,4.366667,-0.005476,05_percent_food,m,Culex tarsalis,no,21.861667,5.108333,-0.006704,19.860000,0.741667,-0.001228
504,191130-09-top,17.060000,1.875000,-0.011828,05_percent_food,f,Culex tarsalis,no,68.208917,0.791667,-0.008254,51.148917,-1.083333,0.003575
